### Run the copy of our database functions

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


| biginteger type of id      | autoincrement type of id |
| ----------- | ----------- |
| <pre>m = DiscordUser<br>e = DiscordEmote<br>g = DiscordGuilds<br>l = LeagueAccount<br>d = DotaAccount<br>a = AFKNote<br>s = mygamerdata</pre>      | <pre>r = RemindersNote<br>t = ToDoNote<br>w = WarningData<br>u = MuteData<br>c = ChannelsAssignmentData<br>sr = serverrules<br>rr = realrule<br>stg = tagsdb</pre>          |

In [1]:
# %run ./utils/database.py
from utils import database as db
import pandas as pd
def get_df(dbclass):
    return pd.read_sql_table(dbclass.__tablename__, db.engine, index_col='id')

### Now our draft things

In [8]:
df = get_df(db.m)

In [6]:
print(sum(get_value(e, 869827100926951465, 'month_array')))

1


In [ ]:
"""
for index, row in df.iterrows():
    value = get_value(e, index, 'month_array')
    df.loc[index, 'month_array'] = value
"""

In [7]:
remove_row(e, 869827100926951465)

In [5]:
with db.session_scope() as ses:
    for row in ses.query(db.sr):
        row.id = 0

In [2]:
db.tg.__table__.drop(db.engine)

In [5]:
def add_column(enginex, table_name, columnx):
    column_name = columnx.compile(dialect=enginex.dialect)
    column_type = columnx.type.compile(enginex.dialect)
    #column_default = column.default.arg
    enginex.execute(f'ALTER TABLE "{table_name}" ADD COLUMN {column_name} {column_type}') #DEFAULT #{column_default}')

column = db.Column('can_make_tags', db.Boolean, default=True)
add_column(db.engine, 'GuildMembers', column)

In [11]:
def drop_column(enginex, table_name, column_name):
    enginex.execute(f'ALTER TABLE "{table_name}" DROP COLUMN {column_name}')

drop_column(engine, 'GuildMembers', 'name')

In [3]:
# options to see the whole table 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# option to reset these things
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')